# Interpretor Data Set

Notebook used to edit the existing dataset for usage in the legalis project. The following can 

In [2]:
from datasets import load_dataset
from bs4 import BeautifulSoup
import re

In [3]:
raw_dataset=load_dataset("LennardZuendorf/openlegaldata-bulk-data", split='train')

Found cached dataset json (C:/Users/lenna/.cache/huggingface/datasets/LennardZuendorf___json/LennardZuendorf--openlegaldata-bulk-data-0d18a44c8ee5464a/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


In [46]:
data = raw_dataset[9]

split_content=data['content'].rsplit(sep="<!--hlIgnoreOn-->Gründe<!--hlIgnoreOff-->")
tmp_soup=BeautifulSoup(split_content[0])
tmp=tmp_soup.get_text().strip()

tmp=re.sub('\n', ' ', tmp)
tmp=' '.join(tmp.split())

print(tmp)



print(tmp)

Tenor Die Beschwerde des Antragstellers gegen den Beschluss des Sozialgerichts Kiel vom 2. August 2022 wird zurückgewiesen. Außergerichtliche Kosten sind auch für das Beschwerdeverfahren nicht zu erstatten.
Die Beschwerde des Antragstellers gegen den Beschluss des Sozialgerichts Kiel vom 2. August 2022 wird zurückgewiesen. Außergerichtliche Kosten sind auch für das Beschwerdeverfahren nicht zu erstatten.


In [49]:
# renaming and removing columns
dataset = raw_dataset.remove_columns(['slug','ecli', 'updated_date', 'created_date'])

#updating columns in nested dict ('court')
def cleaning_courts(court_data):
    del court_data['court']['slug']
    del court_data['court']['city']

    return court_data

dataset = dataset.map(cleaning_courts)

error_count = 0

#splitting content into tenor and reasoning
def splitting_content(data):

    #splitting content into tenor and reasoning
    split_content = data['content'].rsplit(sep="<h2>Gründe</h2>")

    #alternative splitting if first split was not possible
    if len(split_content)!=2:
        split_content=data['content'].rsplit(sep="<!--hlIgnoreOn-->Gründe<!--hlIgnoreOff-->")

    if len(split_content)!=2:

        #settting tenor and reasoning to None if no split was possible
        data['tenor']=None
        data['reasoning']=None

    if len(split_content)==2:

        #cleaning tenor of html, newlines and whitespaces
        tenor_soup=BeautifulSoup(split_content[0])
        tenor=tenor_soup.get_text().strip()
        tenor=re.sub('\n', ' ', tenor)#
        tenor=' '.join(tenor.split())
        data['tenor'] = tenor

        #cleaning reasoning of html, newlines and whitespaces
        reason_soup=BeautifulSoup(split_content[1])
        reason=reason_soup.get_text().strip()
        reason=re.sub('\n', ' ', reason)
        data['reasoning']=' '.join(reason.split())
    
    return data

#splitting content into tenor and reasoning, filter out data without found tenor
dataset = dataset.map(splitting_content)
dataset = dataset.filter(lambda data: data['tenor'] is not None)

Loading cached processed dataset at C:\Users\lenna\.cache\huggingface\datasets\LennardZuendorf___json\LennardZuendorf--openlegaldata-bulk-data-0d18a44c8ee5464a\0.0.0\e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4\cache-3e4c238f57500b9b.arrow


Map:   0%|          | 0/251038 [00:00<?, ? examples/s]

In [ ]:
#

In [50]:
#creating train and test split
dataset = dataset.train_test_split(test_size=0.1)

print(dataset)

In [ ]:
#uploading new dataset into different repository
dataset.push_to_hub("LennardZuendorf/legalis")

Pushing split train to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/3 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/12 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]